# Sentinel-2 NDVI timeseries using Google Earth Engine

This demo gives an overview of calculating a time series of spectral indices using Sentinel-2 data. It uses the Google Earth Engine (GEE) API package for python. This allows to compute large amount of satellite (and other remote sensing) data without having to download the huge data sets. To be able to use this package a GEE account is required.

### Start a GEE session

The first time the "ee" package is used, you need to run ee.Authenticate(). This will open a window on the web where you have to log in with your GEE credentials. This will create an access token that needs to be pasted in the box that appeared below (this needs to be done at the beginning or when the kernel/session is restarted). To start the connection with your GEE account, you run ee.Initialize(). If you have only one project on your GEE account, you do not need to specify more. If you have multiple projects, you can specify with ee.Initialize(project=project-number). To find the project number, you have to log in to GEE online and click on the respective project.

In [34]:
# Load the packages
import ee       # GEE API package
import geemap   # package for interactive plotting->does not work on PyCharm

# Login with the GEE credentials and connect to your account
ee.Authenticate()   # needs to be done once in a while
ee.Initialize()     # starts the connection to your GEE account and allows you to use all the datasets you might have stored there

---

### Define the study areas

For the NDVI time series, we select two different forest types (deciduous broadleaf: "db", evergreen coniferous: "ec") and an agricultural area: "agr" in Switzerland. For each site, we define one point with coordinates and use a buffer to get a circular area of interest. 

In [35]:
# Define the points for each of the AOIs
point_db = ee.Geometry.Point([8.3634171, 47.4788405])
point_ec = ee.Geometry.Point([9.85509283, 46.81539482])
point_agr = ee.Geometry.Point([7.733592499218185, 47.286134852362366])

# Apply a buffer around the defined points. The "buffer number" is the radius of the circle in meters.
aoi_db = point_db.buffer(30)
aoi_ec = point_ec.buffer(30)
aoi_agr = point_agr.buffer(15) # a smaller AOI for the agricultural area to make sure only the field is in the AOI

We can visualize the three areas on a map. The three AOIs are situated where the University of Zürich (and other institutions) have in situ instrumentation: Laegern near Zurich and Baden (deciduous broadleaf), Davos (evergreen coniferous), and Oensingen (agriculture).

In [ ]:
# Initialize the map
Map = geemap.Map()

# Define how the basemap should be displayed
Map.set_center(9.85, 46.82, 13) # coordinates and zoom
Map.add_basemap('TERRAIN')      # can also be "SATELLITE" 

# Define the visualization parameters for the AOI, such as color (as hex-code)
vis_params = {'color': '#d515c5',
              'pointSize': 3,
              'pointShape': 'circle',
              'width': 2,
              'lineType': 'solid',
              'lineType': 'solid'}

# Add the area of interests as polygons to the map
Map.addLayer(aoi_db, vis_params, 'db')
Map.addLayer(aoi_ec, vis_params, 'ec')
Map.addLayer(aoi_agr, vis_params, 'agr')

# Display the map
Map

---

### Data selection

We will work with the Sentinel-2 L2A Bottom-of-Atmosphere reflectance data ImageCollection, as well as the cloud score + ImageCollection for cloud masking. This is an important step to select only valid pixels in the available Sentinel-2 images for the time series analysis. We filter the two Collections by date (e.g., 1 year) and the respective AOIs and add bands from the cloud score + Collection to the Sentinel-2 ImageCollection (by using "linkCollection"). We can then display the number of available images for the respective AOIs in the selected time frame. E.g., during 2024, twice as many images are available for Laegern and Oensingen (Swiss Mittelland) as for Davos (Swiss Alps).

In [36]:
# Define the study period
startTime = '2024-01-01'
endTime = '2024-12-31' 

#-------------------------
# AOI deciduous broadleaf:
#-------------------------
# Filter the Sentinel-2 cloud score+ (for cloud filtering) by date and AOI
db_csp = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')\
    .filter(ee.Filter.bounds(aoi_db))\
    .filter(ee.Filter.date(startTime, endTime))

# Add the cloud score+ bands "cs_cdf" and "cs" to the filtered Sentinel-2 image collection
db_S2= ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filter(ee.Filter.bounds(aoi_db))\
    .filter(ee.Filter.date(startTime, endTime))\
    .linkCollection(db_csp, ['cs_cdf','cs'])

#-------------------------
# AOI evergreen coniferous:
#-------------------------
# Filter the Sentinel-2 cloud score+ (for cloud filtering) by date and AOI
ec_csp = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')\
    .filter(ee.Filter.bounds(aoi_ec))\
    .filter(ee.Filter.date(startTime, endTime))

# Add the cloud score+ bands "cs_cdf" and "cs" to the filtered Sentinel-2 image collection
ec_S2= ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filter(ee.Filter.bounds(aoi_ec))\
    .filter(ee.Filter.date(startTime, endTime))\
    .linkCollection(ec_csp, ['cs_cdf','cs'])

#-------------------------
# AOI agricultural area:
#-------------------------
# Filter the Sentinel-2 cloud score+ (for cloud filtering) by date and AOI
agr_csp = ee.ImageCollection('GOOGLE/CLOUD_SCORE_PLUS/V1/S2_HARMONIZED')\
    .filter(ee.Filter.bounds(aoi_agr))\
    .filter(ee.Filter.date(startTime, endTime))

# Add the cloud score+ bands "cs_cdf" and "cs" to the filtered Sentinel-2 image collection
agr_S2= ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')\
    .filter(ee.Filter.bounds(aoi_agr))\
    .filter(ee.Filter.date(startTime, endTime))\
    .linkCollection(agr_csp, ['cs_cdf','cs'])
              
# Check how many images are in the collection for each AOI
print('Number of images in study period for Laegern:', db_S2.size().getInfo())  
print('Number of images in study period for Davos:', ec_S2.size().getInfo()) 
print('Number of images in study period for Oensingen:', agr_S2.size().getInfo()) 

Number of images in study period for Laegern: 146
Number of images in study period for Davos: 73
Number of images in study period for Oensingen: 144


---

### Cloud and Cloud Shadow Masking

We will now define a function for cloud and cloud shadow masking of the Sentinel-2 images, which we will then apply to each of our filtered ImageCollections from the step before. This will mask pixels in the images that are covered by clouds or cloud shadows. Images with all pixels of the AOI masked will still remain in the ImageCollection.

In [37]:
# Define a function to mask clouds and cloud shadows

def maskCloudsAndShadows(image):

    # get the solar position of each image in the collection
    meanAzimuth = image.get('MEAN_SOLAR_AZIMUTH_ANGLE')
    meanZenith = image.get('MEAN_SOLAR_ZENITH_ANGLE')

    # get the cloud score layer
    clouds_csp = image.select('cs_cdf')  # Pixel quality score (0-1): 0 = cloudy, 1 = clear
    
    # the maximum clear sky probability threshold is set at 60% (change this number to filter more/less images)
    cloudMask_csp = clouds_csp.gte(0.6)
    
    # define potential cloud shadow values
    cloudShadowMask_csp = clouds_csp.gte(0.60).And(clouds_csp.lte(0.80))
    
    # Project shadows from clouds. This step assumes we're working in a UTM projection.
    shadowAzimuth = ee.Number(90).subtract(ee.Number(meanAzimuth))
    
    # shadow distance is tied to the solar zenith angle (minimum shadowDistance is 30 pixel)
    shadowDistance = ee.Number(meanZenith).multiply(0.7).floor().int().max(30)
    
    # With the following algorithm, cloud shadows are projected
    isCloud_csp = cloudMask_csp.directionalDistanceTransform(shadowAzimuth, shadowDistance)
    isCloud_csp = isCloud_csp.reproject(crs=image.select('B2').projection(), scale=100)
    cloudShadow_csp = isCloud_csp.select('distance').mask()
    
    # combine projectedShadows & darkPixel and buffer the cloud shadow
    cloudShadow_csp = cloudShadow_csp.And(cloudShadowMask_csp).focalMax(100, 'circle', 'meters', 1, None)
    
    # combined mask for clouds and cloud shadows
    cloudAndCloudShadowMask_csp = cloudShadow_csp.Or(cloudMask_csp.Not())
    
    # mask spectral bands for clouds and cloudShadows,
    image_out = image.select(['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12'])\
        .updateMask(cloudAndCloudShadowMask_csp.Not())
    
    # adding two additional bands: S2 cloud score plus cloud probability and cloudAndCloudShadowMask
    image_out = image_out.addBands(clouds_csp.multiply(-1).add(1).rename(['cloudProbability']))\
        .addBands(cloudAndCloudShadowMask_csp.rename(['cloudAndCloudShadowMask']))
    
    return image_out    

In [38]:
# Apply the cloud mask function on the ImageCollection for each AOI
db_S2_cf = db_S2.map(maskCloudsAndShadows)
ec_S2_cf = ec_S2.map(maskCloudsAndShadows)
agr_S2_cf = agr_S2.map(maskCloudsAndShadows)

### Filter Images with less then a certain amount if valid (not masked) pixels in the AOI

As mentioned above, images remain in the ImageCollection even if all pixels of the AOI are masked out due to clouds. We can use the number of pixels within the respective AOIs to filter images from the Collection that have fewer than a certain number of pixels. For this, we need to count the number of valid pixels in the AOIs in each image of the ImageCollection and then define a threshold.

In [39]:
# Define a function to count the number of pixels in the AOI
# We can choose from which band the pixels should be counted (matters especially as not all bands have the same spatial resolution and
# hence have different pixel count). We select B8 as we also use this band to calculate the NDVI later. The "scale" defines the spatial 
# resolution we want the pixels counted. 10m is the resolution of B8.

def countValPixels(image, aoi):
    pixels = image.select('B8').reduceRegion(
        reducer=ee.Reducer.count(),
        geometry=aoi,
        scale=10)
    return image.set('valPixels', pixels.get('B8'))

In [40]:
# Use the function above together with ".map()" to add the pixel count as a new property to each of the images in the Collections
# We need to use a "lambda function" as the function we want to use .map() with has two inputs.
db_S2_cf = db_S2_cf.map(lambda image: countValPixels(image, aoi_db))
ec_S2_cf = ec_S2_cf.map(lambda image: countValPixels(image, aoi_ec))
agr_S2_cf = agr_S2_cf.map(lambda image: countValPixels(image, aoi_agr))

We can test whether the function worked by looking at, for example, the first 15 pixel count values of the ImageCollection. This also gives an indication of the threshold we should select. For the db Laegern site, the first few images are not useful as the AOI is masked. But we can assume that the AOI is 28 pixels big, as this is the maximum number we encounter. For the ec Davos site, the maximum number of pixels is 26 pixels even if we selected the same radius for the AOI generation. The agr Oensingen AOI is much smaller, so the maximum pixel count is also smaller. 

In [49]:
print("\nPrinting the 'valPixels' property for the first 15 images:")
count = 1
for image in agr_S2_cf.toList(15).getInfo():
    image_id = image['id']
    pixel_count = image['properties']['valPixels']
    print(f"  - Image ID: {image_id}, Pixel Count: {pixel_count}")
    count += 1


Printing the 'valPixels' property for the first 15 images:
  - Image ID: COPERNICUS/S2_SR_HARMONIZED/20240104T103431_20240104T103427_T32TLT, Pixel Count: 0
  - Image ID: COPERNICUS/S2_SR_HARMONIZED/20240104T103431_20240104T103427_T32TMT, Pixel Count: 0
  - Image ID: COPERNICUS/S2_SR_HARMONIZED/20240109T103329_20240109T103324_T32TLT, Pixel Count: 0
  - Image ID: COPERNICUS/S2_SR_HARMONIZED/20240109T103329_20240109T103324_T32TMT, Pixel Count: 0
  - Image ID: COPERNICUS/S2_SR_HARMONIZED/20240114T103401_20240114T103359_T32TLT, Pixel Count: 0
  - Image ID: COPERNICUS/S2_SR_HARMONIZED/20240114T103401_20240114T103359_T32TMT, Pixel Count: 0
  - Image ID: COPERNICUS/S2_SR_HARMONIZED/20240119T103249_20240119T103252_T32TLT, Pixel Count: 0
  - Image ID: COPERNICUS/S2_SR_HARMONIZED/20240119T103249_20240119T103252_T32TMT, Pixel Count: 0
  - Image ID: COPERNICUS/S2_SR_HARMONIZED/20240124T103321_20240124T103322_T32TLT, Pixel Count: 0
  - Image ID: COPERNICUS/S2_SR_HARMONIZED/20240124T103321_20240124T

In [54]:
# Filter images from the Collection where the pixel number is higher than a threshold
db_S2_cf_filtered = db_S2_cf.filter(ee.Filter.gt('valPixels', 2)) # Laegern threshold of 2 pixels
ec_S2_cf_filtered = ec_S2_cf.filter(ee.Filter.gt('valPixels', 2)) # Davos threshold of 2 pixels
agr_S2_cf_filtered = agr_S2_cf.filter(ee.Filter.gt('valPixels', 1)) # Oensingen threshold of 1 pixel

After filtering images from the Collections where too many pixels in the AOIs are masked due to clouds, we are left with a lot smaller ImageCollections. These we now use to calculate the NDVI and do out time series.

In [55]:
print('Number of images in study period for Laegern:', db_S2.size().getInfo()) 
print('Number of images in study period for Laegern after cloud filterig:', db_S2_cf_filtered.size().getInfo())
print('Number of images in study period for Davos:', ec_S2.size().getInfo()) 
print('Number of images in study period for Davos after cloud filterig:', ec_S2_cf_filtered.size().getInfo())
print('Number of images in study period for Oensingen:', agr_S2.size().getInfo()) 
print('Number of images in study period for Oensingen after cloud filterig:', agr_S2_cf_filtered.size().getInfo())

Number of images in study period for Laegern: 146
Number of images in study period for Laegern after cloud filterig: 40
Number of images in study period for Davos: 73
Number of images in study period for Davos after cloud filterig: 32
Number of images in study period for Oensingen: 144
Number of images in study period for Oensingen after cloud filterig: 24


---

### Calculate the spectral index NDVI

**Normalized difference vegetation index - NDVI**<br>
The normalized difference vegetation index (NDVI) was developed by Tucker et al. in 1970tes and is widely used to estimate the green vegetation$^{[1]}$. The index normalizes green leaf scattering in NIR with chlorophyll absorption in red wavelengths and uses the following formula$^{[2]}$:<br>

$NDVI := \frac {NIR - red}{NIR + red} = \frac {B08 - B04}{B08 + B04}$ <br>

In [56]:
# Define a function to add the NDVI as a new band to each image
def calcNDVI(image):
    return image.addBands(image.normalizedDifference(['B8', 'B4']).rename('NDVI'))

In [62]:
# Use the NDVI function and ".map()" to add the index as a new band to each image in the filtered Collection
db_S2_ndvi = db_S2_cf_filtered.map(calcNDVI)
ec_S2_ndvi = ec_S2_cf_filtered.map(calcNDVI)
agr_S2_ndvi = agr_S2_cf_filtered.map(calcNDVI)

### Create time series

*Literature*<br>
$^{[1]}$: Tucker, C.J., 1979. Red and photographic infrared linear combinations for monitoring vegetation. Remote Sensing of Environment 8, 127–150.<br>
$^{[2]}$: https://custom-scripts.sentinel-hub.com/sentinel-2/ndvi/<br>